In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
import copy
import torch
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

In [2]:
    data_dir = 'C:/Users/joey3/Desktop/Apnea'

    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ]),
        'val': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ]),
    }

    image_datasets = {
        x: datasets.ImageFolder(
            os.path.join(data_dir, x), 
            transform=data_transforms[x],
        ) 
        for x in ['train', 'val']
    }
    dataloaders = {
        x: torch.utils.data.DataLoader(
            image_datasets[x], 
            batch_size=8,
            shuffle=True, 
            num_workers=8,
        )
        for x in ['train', 'val']
    }
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
    class_names = image_datasets['train'].classes

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [4]:
def test_model(model):
    model.eval()   # Set model to evaluate mode

    running_count, running_corrects = 0, 0
    test_df = pd.read_csv('../resources/File_test.csv')
    # Iterate over testing patients
    for file in test_df['file']:
        image_datasets = datasets.ImageFolder(
            f'{data_dir}/test/{file}', 
            transform=data_transforms['val'],
        ) 
        dataloaders = torch.utils.data.DataLoader(
            image_datasets, 
            batch_size=8,
            shuffle=True, 
            num_workers=8,
        )

        # Iterate over data.
        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)

            # statistics
            running_count += len(preds)
            running_corrects += torch.sum(preds == labels.data)

    epoch_acc = running_corrects.double() / running_count
    print('Testing Acc: {:.4f}'.format(epoch_acc))

# Finetuning 

In [5]:
# ResNet18
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = torch.nn.Linear(num_ftrs, 2)
model_ft = model_ft.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)
torch.save(model_ft, '../resources/resnet.pt')
test_model(model_ft)

Epoch 0/19
----------
train Loss: 0.4219 Acc: 0.8176
val Loss: 0.4755 Acc: 0.7915

Epoch 1/19
----------
train Loss: 0.3760 Acc: 0.8405
val Loss: 0.5990 Acc: 0.7741

Epoch 2/19
----------
train Loss: 0.3488 Acc: 0.8530
val Loss: 0.6383 Acc: 0.6079

Epoch 3/19
----------
train Loss: 0.3259 Acc: 0.8602
val Loss: 0.4040 Acc: 0.8244

Epoch 4/19
----------
train Loss: 0.2971 Acc: 0.8753
val Loss: 0.6678 Acc: 0.7706

Epoch 5/19
----------
train Loss: 0.2654 Acc: 0.8889
val Loss: 0.4020 Acc: 0.8273

Epoch 6/19
----------
train Loss: 0.2348 Acc: 0.9056
val Loss: 0.4716 Acc: 0.8250

Epoch 7/19
----------
train Loss: 0.1528 Acc: 0.9447
val Loss: 0.3919 Acc: 0.8496

Epoch 8/19
----------
train Loss: 0.1252 Acc: 0.9590
val Loss: 0.4057 Acc: 0.8486

Epoch 9/19
----------
train Loss: 0.1140 Acc: 0.9639
val Loss: 0.4158 Acc: 0.8496

Epoch 10/19
----------
train Loss: 0.1020 Acc: 0.9685
val Loss: 0.4423 Acc: 0.8478

Epoch 11/19
----------
train Loss: 0.0922 Acc: 0.9746
val Loss: 0.4339 Acc: 0.8482

Ep

In [5]:
# SqueezeNet
model_ft_2 = models.squeezenet1_0(pretrained=True)
model_ft_2.classifier[1] = torch.nn.Conv2d(512, 2, kernel_size=(1,1), stride=(1,1))
model_ft_2.num_classes = 2
model_ft_2 = model_ft_2.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer_ft = torch.optim.SGD(model_ft_2.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
model_ft_2 = train_model(model_ft_2, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)
torch.save(model_ft_2, '../resources/squeezenet.pt')
test_model(model_ft_2)

Epoch 0/19
----------
train Loss: 0.4656 Acc: 0.7923
val Loss: 0.4248 Acc: 0.8283

Epoch 1/19
----------
train Loss: 0.4261 Acc: 0.8151
val Loss: 0.4394 Acc: 0.8117

Epoch 2/19
----------
train Loss: 0.4127 Acc: 0.8261
val Loss: 0.4093 Acc: 0.8334

Epoch 3/19
----------
train Loss: 0.4009 Acc: 0.8281
val Loss: 0.4540 Acc: 0.8279

Epoch 4/19
----------
train Loss: 0.3946 Acc: 0.8332
val Loss: 0.3843 Acc: 0.8414

Epoch 5/19
----------
train Loss: 0.3902 Acc: 0.8310
val Loss: 0.3910 Acc: 0.8388

Epoch 6/19
----------
train Loss: 0.3843 Acc: 0.8358
val Loss: 0.3857 Acc: 0.8367

Epoch 7/19
----------
train Loss: 0.3575 Acc: 0.8466
val Loss: 0.3695 Acc: 0.8484

Epoch 8/19
----------
train Loss: 0.3526 Acc: 0.8505
val Loss: 0.3695 Acc: 0.8480

Epoch 9/19
----------
train Loss: 0.3518 Acc: 0.8502
val Loss: 0.3688 Acc: 0.8482

Epoch 10/19
----------
train Loss: 0.3491 Acc: 0.8518
val Loss: 0.3693 Acc: 0.8474

Epoch 11/19
----------
train Loss: 0.3482 Acc: 0.8515
val Loss: 0.3666 Acc: 0.8494

Ep

In [6]:
# GoogLeNet
model_ft_3 = models.googlenet(pretrained=True)
num_ftrs = model_ft_3.fc.in_features
model_ft_3.fc = torch.nn.Linear(num_ftrs, 2)
model_ft_3 = model_ft_3.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer_ft = torch.optim.SGD(model_ft_3.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
model_ft_3 = train_model(model_ft_3, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)
torch.save(model_ft_3, '../resources/googlenet.pt')
test_model(model_ft_3)

Epoch 0/19
----------
train Loss: 0.4673 Acc: 0.7923
val Loss: 0.4150 Acc: 0.8257

Epoch 1/19
----------
train Loss: 0.4102 Acc: 0.8266
val Loss: 0.4580 Acc: 0.8340

Epoch 2/19
----------
train Loss: 0.3835 Acc: 0.8384
val Loss: 0.3873 Acc: 0.8422

Epoch 3/19
----------
train Loss: 0.3649 Acc: 0.8473
val Loss: 0.4023 Acc: 0.8226

Epoch 4/19
----------
train Loss: 0.3446 Acc: 0.8582
val Loss: 0.3708 Acc: 0.8416

Epoch 5/19
----------
train Loss: 0.3272 Acc: 0.8662
val Loss: 0.4430 Acc: 0.7942

Epoch 6/19
----------
train Loss: 0.3091 Acc: 0.8713
val Loss: 0.4075 Acc: 0.8482

Epoch 7/19
----------
train Loss: 0.2434 Acc: 0.9026
val Loss: 0.3630 Acc: 0.8492

Epoch 8/19
----------
train Loss: 0.2145 Acc: 0.9151
val Loss: 0.3644 Acc: 0.8492

Epoch 9/19
----------
train Loss: 0.1971 Acc: 0.9224
val Loss: 0.3816 Acc: 0.8435

Epoch 10/19
----------
train Loss: 0.1777 Acc: 0.9326
val Loss: 0.3793 Acc: 0.8521

Epoch 11/19
----------
train Loss: 0.1630 Acc: 0.9363
val Loss: 0.3928 Acc: 0.8541

Ep